### Day 15 - Beacon Exclusion Zone
##### Part 1 - Beaconless positions on y = 2,000,000

Find positions in row 2,000,000 that cannot contain a beacon according to the rules of this challenge.

In [467]:
def get_data(input_file):
    with open(input_file, "r") as f:
        lines = [a for b in f.read().strip().split("\n") for c in b.split(":") for a in c.split("=")]
    process_data = list(zip(list(map( lambda a : int(a.split(',')[0]), lines[1::3])), \
                          list(map( lambda a : int(a), lines[2::3]))))
    return process_data[::2], process_data[1::2]

dist = lambda ax, ay, bx, by : abs(ax - bx) + abs(ay - by)

# Pretty print for our Part 1 result list - also for debugging.
def ppres(r):
    print("\t210123456789a123456789b123")
    print("\t" + "".join(['.' if c == 0 else '#' for c in r]), len(r))


In [468]:
def part1(sensors, beacons, target, candidates, beacons_on_target):
    min_x = min([a[0] - dist(*a,*beacons[i]) for i,a in enumerate(sensors)])
    max_x = max([a[0] + dist(*a,*beacons[i]) for i,a in enumerate(sensors)])
    res = [0 for _ in range(min_x, max_x + 1)]
    
    for candidate in candidates:
            # same line behavior
        if candidate[0][1] == target:
            offset = candidate[0][0] - min_x - candidate[2]
            res = res[:offset] + [1 for _ in range((2 * candidate[2]) + 1)] + res[offset + (2 * candidate[2] + 1):]

            # above/under
        else:
            diff = max(candidate[0][1], target) - min(candidate[0][1], target)
            remaining = candidate[2] - diff
            if remaining == 0:
                res[candidate[0][0] - min_x] = 1
            else:
                offset = candidate[0][0] - min_x - remaining
                res = res[:offset] + [1 for _ in range((2 * remaining) + 1)] + res[offset + (2 * remaining + 1):]
                
    # count impossible spaces minus known beacons 
    return res.count(1) - sum([res[b[0] - min_x] for b in beacons_on_target])

In [469]:
targ = 2_000_000
input_sensors, input_beacons =  get_data('aoc_15_input.txt')
candidate_sensors = list(filter( lambda a: targ - a[2] <= a[0][1] <= targ + a[2] , list(zip(input_sensors, input_beacons, [dist(*input_beacons[i], *input_beacons[i]) for i in range(len(input_sensors))] ))))
target_beacons    = list(set([b for b in input_beacons if b[1] == targ]))
print(f"Part 1: For y = 2,000,000 there are {part1(input_sensors, input_beacons, targ, candidate_sensors, target_beacons)} positions which cannot contain a beacon.")

Part 1: For y = 2,000,000 there are 0 positions which cannot contain a beacon.


##### Part 2 - Lone Unfound Beacon

Find tuning frequency ( x * 4,000,000 + y ) of the last beacon, which is in the only space untouched by current assortment of beacons.


In [470]:
# See hyper-neutrino video
# https://www.youtube.com/watch?v=OG1QwJ2RKsU

def part2(data, lb, ub):
    for Y in range(lb, ub + 1):
        intervals = []
        for sx, sy, bx, by, d in data:
            o = d - abs(sy - Y)
            if o < 0:    continue
            intervals.append((sx - o, sx + o))
        intervals.sort()
        q = []
        for lo, hi in intervals:
            if not q:
                q.append([lo, hi])
                continue
            qlo, qhi = q[-1]
            if lo > qhi + 1:        
                return ((qhi + 1) * 4_000_000) + Y
            q[-1][1] = max(qhi, hi)
    return -1

In [471]:
from random import shuffle
import timeit
start_time = timeit.default_timer()

# ~~~ Times, improvements
#  * original                                      52.7s, 51.4s
#  * memoize manhatten distance                 -> 26.7s
#  * ignore negative intervals, bound intervals -> 29.2s, 31.4s
#  * revert last change                         -> 23.2s, 22.8s
#   okay so ... faster to just iterate thru final intervals
#   than try to massage them to be nicer

#  * after some misc cleanup avg is 27.5s
#  * moving parameters around and cleanup       -> 21.4s, 22.1s, 22.1s, 23.6s
#  * can end very minutely earlier              -> 21.4s, 22.7s
#
# Pretty happy with this!

# UPPER BOUND = 4_000_000
# seems likelier to be in the middle that near the edge so
# check edges last - we only are in them 1/8 + 1/8 = 1/4 of the time
# so we partition the space into 8 ranges
# because getting a 1.5s hit some of the time is worth going from 22s to 30s sometimes

#   0 : 1.6s
#   1 : 6.9s
#   2 : 9.8s
#   3 : 14.7s
#   4 : 19.3s
#   5 : 23.4s  -- this is where we were hitting for all runs previously (since the position is fixed in the range)
#   6 : 27.3s  <rare>
#   7 : 33.6s  <rare>

avg_time = sum([1.6, 6.9, 9.8, 14.7, 19.3, 23.4, 27.3, 33.6]) / 8
control_time = sum([21.4, 22.1, 22.1, 23.6, 21.4, 22.7]) / 6
print(f"Expected average time for randomizing order:  {avg_time:6.2f}")

# %%%%%%% COME BACK AND FIX LATER (bayes thm, probability?)
#print(f"Expected avg time for random with edges last: {(sum([1.6, 6.9, 9.8, 14.7, 19.3, 23.4]) / 6) + sum([27.3, 33.6]) / 16:6.2f}")
# %%%%%%% COME BACK AND FIX LATER

print(f"Control average time (0 -> 4,000,000):        {control_time:6.2f}")
print(f"Expected value for randomization:             {control_time - avg_time:6.2f}")
#print(f"EV for edge last randomization:               {avg_time:6.2f}")

boundary_slices = [(500_000 * i, 500_000 * (i+1)) for i in range(1, 7)]
shuffle(boundary_slices)
boundary_slices += [(3_500_000, 4_000_000), (0, 500_000)]


p2_data = [[s[0], s[1], input_beacons[i][0], input_beacons[i][1], dist(s[0], s[1], input_beacons[i][0], input_beacons[i][1])] for i,s in enumerate(input_sensors)]

for i, (lower, upper) in enumerate(boundary_slices):
    res = part2(p2_data, lower, upper)
    if res != -1:
        print(f"\n\t\t\tPart 2: The tuning frequency of the final distress beacon is {res}.\n")
        print(f"This run, the answer was in the {i+1}{'th' if i+1 > 3 else 'st' if i == 0 else 'nd' if i == 1 else 'rd'} slice of size 500,000.")
        print(f"This saved approximately {round(control_time, 2) - (round(timeit.default_timer() - start_time, 2)) : 6.2f}")
        break
            


Expected average time for randomizing order:   17.07
Control average time (0 -> 4,000,000):         22.22
Expected value for randomization:               5.14

			Part 2: The tuning frequency of the final distress beacon is 13081194638237.

This run, the answer was in the 1st slice of size 500,000 slices.
This saved approximately  20.64
